## Collect data

Read in names of basketballers and check their name is 2 tokens exactly.

In [2]:
import pandas as pd
from pathlib import Path
import torch
import transformer_lens

In [ ]:
# data_dir = Path("data")
# names_file = data_dir / "names.txt"
# all_new_names = []

# # Iterate over all csv files in the data directory
# for csv_path in data_dir.glob("*.csv"):
#     # Read the csv file, assuming no header
#     df = pd.read_csv(csv_path, header=None)
#     # Get the second column (index 1) and drop any missing values
#     names_from_csv = df.iloc[:, 1].dropna().tolist()
#     filtered_names = [name for name in names_from_csv if isinstance(name, str) and name.count(' ') == 1]
#     all_new_names.extend(filtered_names)

# # Append the collected names to the names.txt file
# if all_new_names:
#     with open(names_file, "a") as f:
#         # Ensure we start on a new line if the file isn't empty
#         if names_file.stat().st_size > 0:
#             f.write("\n")
#         f.write("\n".join(all_new_names))
#     print(f"Appended {len(all_new_names)} names to {names_file}.")
# else:
#     print("No new names found in CSV files to append.")

Appended 956 names to data/names.txt.


In [4]:
names_file = Path("convex_sets/data/names.txt")
with open(names_file, 'r') as f:
    names = f.read().strip().split('\n')
    print(names)

['Michael Jordan', 'Kobe Bryant', 'LeBron James', 'R. Kelly', 'Kareem Abdul-Jabbar', 'Michael Crichton', 'Magic Johnson', 'Edi Rama', "Shaquille O'Neal", 'Larry Bird', 'Carl Lewis', 'Kevin Durant', 'Stephen Curry', 'Manny Pacquiao', 'Nick Nolte', 'Wilt Chamberlain', 'Bill Russell', 'Charles Barkley', 'Tim Duncan', 'Giannis Antetokounmpo', 'Dirk Nowitzki', 'Yao Ming', 'Jüri Ratas', 'Dennis Rodman', 'Mahershala Ali', 'Jim Thorpe', 'Dwyane Wade', 'James Naismith', 'Dražen Petrović', 'Karl Malone', 'George Floyd', 'Hakeem Olajuwon', 'Allen Iverson', 'Derrick Rose', 'Tony Parker', 'Scottie Pippen', 'Jackie Joyner-Kersee', 'Tom Selleck', 'Luka Dončić', 'Chris Paul', 'Pau Gasol', 'Russell Westbrook', 'Kyrie Irving', 'Paul Robeson', 'James Harden', 'Kevin Garnett', 'Bob Beamon', 'Steve Nash', 'Arvydas Sabonis', 'Jerry West', 'Carmelo Anthony', 'Julius Erving', 'Klay Thompson', 'John Stockton', 'Nikola Jokić', 'Paul Pierce', 'Marion Jones', 'Kawhi Leonard', 'Anthony Davis', 'Dwight Howard', 'Vl

In [5]:
# Load the Pythia-2.8b model
# This will download the model weights if they are not already cached.
model = transformer_lens.HookedTransformer.from_pretrained(
    "EleutherAI/pythia-2.8b",
    device="cuda" if torch.cuda.is_available() else "cpu"
)

print("Pythia-2.8b model loaded successfully.")

Loaded pretrained model EleutherAI/pythia-2.8b into HookedTransformer
Pythia-2.8b model loaded successfully.


In [6]:
# A list to store names that consist of exactly two tokens
two_token_names = []

for name in names:
    # Tokenize the name. The result includes a batch dimension.
    tokens = model.to_tokens(name)

    # Check if the number of tokens (the length of the second dimension) is 3
    # (<soe> token, first name, last name)
    if tokens.shape[1] == 3:
        two_token_names.append(name)
    else:
        print(f"Name '{name}' is represented by {tokens}.")

print(f"Found {len(two_token_names)} names that are exactly two tokens long.")
print(two_token_names)

Name 'Kobe Bryant' is represented by tensor([[    0,    44,  8594, 33276]]).
Name 'LeBron James' is represented by tensor([[    0,  4015, 39674,  5490]]).
Name 'R. Kelly' is represented by tensor([[    0,    51,    15, 14943]]).
Name 'Kareem Abdul-Jabbar' is represented by tensor([[    0,    44,   609,   358, 39509,    14,    43,   357,  2009]]).
Name 'Michael Crichton' is represented by tensor([[    0, 18744,   330, 32246,   251]]).
Name 'Edi Rama' is represented by tensor([[   0, 3996,   74,  416, 2902]]).
Name 'Shaquille O'Neal' is represented by tensor([[    0,  2809, 35471,  4002,   473,     8,  6560,   267]]).
Name 'Larry Bird' is represented by tensor([[    0, 20678,   610, 25080]]).
Name 'Kevin Durant' is represented by tensor([[    0, 40773, 17065,   386]]).
Name 'Manny Pacquiao' is represented by tensor([[    0,    46, 10059,  9620,   371, 22728]]).
Name 'Nick Nolte' is represented by tensor([[    0, 31853,   427,   311,   442]]).
Name 'Wilt Chamberlain' is represented by ten

In [ ]:
basketball_players = []
print("Completions for non-basketball players:")
print("---------------------------------------")

for name in ["Michael Jordan"]:
    prompt = f"Fact: {name} is known for playing the sport of "
    
    # Generate a completion for the prompt
    # We generate a few tokens and set temperature to 0 for deterministic output
    completion = model.generate(
        prompt, 
        max_new_tokens=5, 
        temperature=1,
        verbose=True
    )
    
    # Extract just the generated part of the text
    completion_text = completion[len(prompt):].strip()
    
    # Check if the completion is 'basketball' (case-insensitive)
    if completion_text.lower().startswith("basketball"):
        basketball_players.append(name)
    else:
        print(completion)
        print(completion_text)

print("\n---------------------------------------")
print(f"Found {len(basketball_players)} names that completed with 'basketball':")
print(basketball_players)

Completions for non-basketball players:
---------------------------------------


100%|█████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.51it/s]

Fact: Michael Jordan is known for playing the sport of 
metaphor.

---------------------------------------
Found 0 names that completed with 'basketball':
[]


In [ ]:
import itertools
import random

# Use the list of confirmed basketball players from the previous step
# Extract first and last names
first_names = list(set([name.split(' ')[0] for name in names]))
last_names = list(set([name.split(' ')[1] for name in names]))

# Create all possible name combinations
all_combinations = [" ".join(p) for p in itertools.product(first_names, last_names)]

# Filter out the names that actually exist
real_names_set = set(names)
fake_names = [name for name in all_combinations if name not in real_names_set]

print(f"Generated {len(fake_names)} fake names to test.")

non_basketball_fake_players = []
non_basketball_completions = []

# Run basketball detection on the fake names
for name in fake_names:
    prompt = f"{name} is famous for playing the sport of "
    
    completion = model.generate(
        prompt, 
        max_new_tokens=5, 
        temperature=0,
        verbose=False
    )
    
    completion_text = completion[len(prompt):].strip()
    
    # Keep the names that do NOT complete with "basketball"
    if not completion_text.lower().startswith("basketball"):
        non_basketball_fake_players.append(name)
        non_basketball_completions.append(completion)

print(f"\nFound {len(non_basketball_fake_players)} fake names that are not associated with basketball.")
print("A sample of non-basketball completions for fake names:")
print("----------------------------------------------------")

# Define the sample size
sample_size = min(20, len(non_basketball_completions))

# Print a random sample of the results
if sample_size > 0:
    for i in random.sample(range(len(non_basketball_completions)), sample_size):
        print(non_basketball_completions[i])